En los siguiente apartados se muestra el proceso que realizo para reducir el tamaño del dataset final con el motivo de evitar problemas dentro de la plataforma de render

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ast

Primeramente se carga el documento de credits y movies_dataset para analizar y unir la informacion de ambos datasets en un solo 

In [3]:
peliculas= pd.read_csv('movies_dataset.csv', dtype={"overview": str})

C:\Users\Jesus\AppData\Local\Temp\ipykernel_28428\3808256760.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  peliculas= pd.read_csv('movies_dataset.csv', dtype={"overview": str})


In [4]:
creditos=pd.read_csv('credits.csv')

Se identifican las celdas similares, esto permitira que se pueda hacer un merge como si fuera un join en SQL

In [5]:
peliculas['id']=peliculas['id'].astype(str) #se convierte a string ambas variables para evitar valores flotantes que eviten la union
creditos['id']=creditos['id'].astype(str)

se hace la union de los dataset por medio de un inner join sobre la columna del mismo nombre

In [6]:
df_unido=pd.merge(peliculas,creditos, on='id', how='inner')

Se muestran todas las columnas del nuevo dataframe que contiene ambos datasets

In [7]:
df_unido.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'cast', 'crew'],
      dtype='object')

Debido a que las columnas de datos categoricos vienen como formato de texto, es necesario identificar y transformar esas columnas a valores que pueda manejar python, como pueden ser, diccionarios, listas, tuplas ,etc para ello se utiliza la libreria ast y literal_eval

In [8]:
df_unido['cast']=df_unido['cast'].apply(ast.literal_eval)

se crea una nueva columna con el nombre actores en el cual se sacaran unicamente los valores de la clave name que contenian los diccionarios de la columna cast, toda la informacion restante es innecesaria

In [9]:
df_unido['actores']=df_unido['cast'].apply(lambda lista: [d['name'] for d in lista])

A continuacion se realiza lo mismo para las columnas crew y genres, ademas, tambien se generan las nuevas columnas "puesto", "nombre" y "generos"

In [9]:
df_unido['crew']=df_unido['crew'].apply(ast.literal_eval)

In [10]:
df_unido['puesto']=df_unido['crew'].apply(lambda lista: [d['job'] for d in lista])

In [11]:
df_unido['nombre']=df_unido['crew'].apply(lambda lista: [d['name'] for d in lista])

In [12]:
df_unido['genres']=df_unido['genres'].apply(ast.literal_eval)

In [13]:
df_unido['generos']=df_unido['genres'].apply(lambda lista: [d['name'] for d in lista])

Debido a que algunas columnas cuentan con listas vacias o valores nulos, etc, que evitaban el uso del metodo anterior, se desarrollo una funcion que permite extraer la informacion necesaria de las colunmas a las cuales no se les podia aplicar el metodo anterior

In [10]:
def extract_val2(row):
    # Verificamos si 'row' es un diccionario o un string
    if isinstance(row, str):
        try:
            row = ast.literal_eval(row)  # Convertimos el string a objeto de Python
        except (ValueError, SyntaxError): #si existen errores de conversion retornamos None para cada columna nueva
            return pd.Series([None, None])
    
    # Verificamos si 'row' es un diccionario y tiene las claves que necesitamos
    if isinstance(row, dict):
        name = row.get('name')  #se extrae el valor de la clave nombre
        id_ = row.get('id')  #se extrae el valor de la clave id
        return pd.Series([name, id_])

    return pd.Series([None, None]) #en caso de encontrar elementos nulos, regresa una serie con None para cada nueva columna creada

Se generan las nuevas columnas y se aplica el metodo a la columna de belongs to collection

In [11]:
df_unido['belongs_to_collection']=pd.DataFrame(df_unido['belongs_to_collection'])

In [12]:
df_unido[['collection', 'id_collec']] = df_unido['belongs_to_collection'].apply(extract_val2)

In [17]:
df_unido['production_companies']=pd.DataFrame(df_unido['production_companies'])

In [18]:
df_unido['spoken_languages']=pd.DataFrame(df_unido['spoken_languages']) 

A continuacion se crea una nueva funcion dado que la anterior devolvia valores completamente nulos y debido a que las claves contenian nombres diferentes por lo cual el metodo previo no se podia aplicar (no se logro generar una unica funcion que fuera aplicable a todas las columnas independentemente del tipo de valores que contenian las columnas)

In [ ]:
def extract_val(row):
    if isinstance(row, str):
        try:
            data_list=ast.literal_eval(row) # Convertimos el string a objeto de Python
            if isinstance(data_list, list):
                ids=[d['id'] for d in data_list if 'id' in d] #se busca el key id en los diccionarios y se extrae el valor 
                names=[d['name'] for d in data_list if 'name' in d] #se busca el key name en los diccionarios y se extrae el valor 
                return pd.Series([names,ids])  #se generan las columnas en formato serie de pandas
        except(ValueError, SyntaxError):  #en caso de que existan errores de conversion o existan valores nulos se ignoran y se devuelve una serie de panda con valores None None
            pass
    return pd.Series([None,None])

Se aplica la funcion a las columnas de spoken_languages y production_companies

In [20]:
df_unido[['name_doblaje', 'iso_639_1']] = df_unido['spoken_languages'].apply(extract_val)


In [21]:
df_unido[['name_compañia', 'id_compañia']] = df_unido['production_companies'].apply(extract_val)

Se eliminan las columnas que contienen los diccionarios, asi como las columnas innecesarias para lograr los endpoints o el modelo de ML

In [24]:
df_unido.drop(['adult','belongs_to_collection','genres','homepage','imdb_id','original_title','poster_path','production_companies','production_countries','spoken_languages','status','video','cast','crew','id_collec','iso_639_1','id_compañia'],axis=1, inplace=True)

se eliminan dos columnas mas luego de revisar su informacion

In [29]:
df_unido.drop(['tagline','runtime'],axis=1,inplace=True)

Se genera un nuevo documento que inicialmente se pensaba seria el final

In [43]:
df_unido.to_csv('dataset_final.csv',index=False)

Se lee el nuevo dataset creado

In [44]:
df=pd.read_csv('dataset_final.csv')

Se eliminan los titulos y fechas de estreno con valores vacions o nulos

In [53]:
df.dropna(subset=['title'],inplace=True)

In [55]:
df.dropna(subset=['release_date'],inplace=True)

Se filtran todas las columnas donde el idioma de doblaje no contenga el idioma ingles, esto es para disminuir el tamaño del dataset y se considera ese idioma ya que es el mas hablado muldialmente

In [58]:
df_filtrado = df[df['name_doblaje'].str.contains('English', case=False, na=False)]

Se renombran las columnas para evitar conflictos de nombres en posteriores procesos

In [66]:
df_filtrado.rename(columns={'puesto':'puestos_produccion','nombre':'nombre_produccion','name_doblaje':'doblajes','name_compañia':'companies'}, inplace=True)

C:\Users\Jesus\AppData\Local\Temp\ipykernel_36324\943177078.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado.rename(columns={'puesto':'puestos_produccion','nombre':'nombre_produccion','name_doblaje':'doblajes','name_compañia':'companies'}, inplace=True)


Se analiza el resultado del dataframe filtrado se observa que casi no existen registros o columnas con elementos nulos

In [70]:
df_filtrado.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28771 entries, 0 to 45537
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   budget              28771 non-null  int64  
 1   id                  28771 non-null  int64  
 2   original_language   28764 non-null  object 
 3   overview            28690 non-null  object 
 4   popularity          28771 non-null  float64
 5   release_date        28771 non-null  object 
 6   revenue             28771 non-null  float64
 7   title               28771 non-null  object 
 8   vote_average        28771 non-null  float64
 9   vote_count          28771 non-null  float64
 10  actores             28771 non-null  object 
 11  puestos_produccion  28771 non-null  object 
 12  nombre_produccion   28771 non-null  object 
 13  generos             28771 non-null  object 
 14  collection          3144 non-null   object 
 15  doblajes            28771 non-null  object 
 16  companies

Se generan 3 nuevas columnas para poder realizar los endpoints de la api que se subira a render 

In [76]:
df_filtrado['release_date'] = pd.to_datetime(df_filtrado['release_date'], errors='coerce')

C:\Users\Jesus\AppData\Local\Temp\ipykernel_36324\379485518.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado['release_date'] = pd.to_datetime(df_filtrado['release_date'], errors='coerce')


Las columnas son dia, mes y año

In [77]:
df_filtrado['mes'] = df_filtrado['release_date'].dt.month
df_filtrado['dia'] = df_filtrado['release_date'].dt.day
df_filtrado['año'] = df_filtrado['release_date'].dt.year

C:\Users\Jesus\AppData\Local\Temp\ipykernel_36324\3926080634.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado['mes'] = df_filtrado['release_date'].dt.month
C:\Users\Jesus\AppData\Local\Temp\ipykernel_36324\3926080634.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado['dia'] = df_filtrado['release_date'].dt.day
C:\Users\Jesus\AppData\Local\Temp\ipykernel_36324\3926080634.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

Al analizar a detalle se determina que la columna dia no servira para lo que se pretende realizar (determinar el numero de peliculas estrenadas los dias "lunes, martes, miercoles, ect) por lo cual se elimina la columna

In [79]:
df_filtrado.drop(['dia'], axis=1 , inplace=True)

C:\Users\Jesus\AppData\Local\Temp\ipykernel_36324\1821171024.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado.drop(['dia'], axis=1 , inplace=True)


In [96]:
df_filtrado.rename(columns={'año':'release_year'}, inplace=True)

C:\Users\Jesus\AppData\Local\Temp\ipykernel_36324\820308054.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado.rename(columns={'año':'release_year'}, inplace=True)


Se crea la columna return que es la division entre revenue y budget

In [98]:
df_filtrado['return'] = df_filtrado['revenue'] / df_filtrado['budget']

C:\Users\Jesus\AppData\Local\Temp\ipykernel_36324\3822242372.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado['return'] = df_filtrado['revenue'] / df_filtrado['budget']


In [101]:
df_filtrado['return']

0        12.451801
1         4.043035
2              NaN
3         5.090760
4              inf
           ...    
45530          NaN
45531          NaN
45532          NaN
45535          NaN
45537          NaN
Name: return, Length: 28771, dtype: float64

En caso de existir valores infinitos positivos o negativos estos son reemplazados por 0 y en caso de haber valores vacios, estos tambien se rellenan con 0

In [102]:
df_filtrado['return']=df_filtrado['return'].fillna(0).replace([float('inf'), -float('inf')],0)

C:\Users\Jesus\AppData\Local\Temp\ipykernel_36324\2021111667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado['return']=df_filtrado['return'].fillna(0).replace([float('inf'), -float('inf')],0)


Se crea el nuevo documento con toda la informacion necesaria y listo para ser consumido por las apis y el modelo de machine learning

In [105]:
df_filtrado.to_csv('dataset.csv',index=False)